In [1]:
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from typing import List, Tuple
from pytorch3d.renderer.cameras import get_screen_to_ndc_transform, get_ndc_to_screen_transform
from src.util.cameras import PatchPerspectiveCameras, get_patch_ndc_to_ndc_transform, get_ndc_to_patch_ndc_transform

In [2]:
# repeat sample image batch_size times with different patches of different sizes
image_size = [(256, 256), (128, 64), (64, 128), (32, 32)]
patch_size = [(64, 64), (32, 32), (16, 16), (8, 8)]

# wrt the center of the full image in pixels
patch_center = [(0, 0), (0, 0), (0, 0), (0, 0)]

cam_kwargs = {
    "znear": 0.0,
    "zfar": 80.0,
    "focal_length": 1.0,
    "principal_point": ((0.0, 0.0),),
    "R": torch.eye(3).unsqueeze(0),
    "T": torch.zeros(1, 3),
    "device": "cpu",
    "in_ndc": False,
    "image_size": image_size
    }

cam = PatchPerspectiveCameras(**cam_kwargs)

In [3]:
len(cam.get_ndc_camera_transform(**cam_kwargs))

4

In [4]:
world_to_patch_ndc_transform = cam.get_patch_projection_transform(patch_size, patch_center, **cam_kwargs)

In [5]:
patch_ndc_to_world_transform = world_to_patch_ndc_transform.inverse()

In [6]:
X_patch_ndc = torch.tensor([[0.5, 0.5, 1.0], [0.25, 0.25, 1.0], [0.75, 0.75, 1.0], [1., 1., 1.0]])
X_world = patch_ndc_to_world_transform.transform_points(X_patch_ndc)

In [7]:
world_to_patch_ndc_transform = cam.get_patch_projection_transform(patch_size, patch_center, **cam_kwargs)

In [8]:
X_patch_ndc = world_to_patch_ndc_transform.transform_points(X_world)  

In [22]:
# repeat sample image batch_size times with different patches of different sizes
image_size = [(256, 256)]
patch_size = [(256, 256)]

# wrt corner of the full image in pixels
patch_center = [(128, 128)]

cam_kwargs = {
    "znear": 0.0,
    "zfar": 80.0,
    "focal_length": 1.0,
    "principal_point": ((0.0, 0.0),),
    "R": torch.eye(3).unsqueeze(0),
    "T": torch.zeros(1, 3),
    "device": "cpu",
    "in_ndc": False,
    "image_size": image_size
    }

cam = PatchPerspectiveCameras(**cam_kwargs)

In [23]:
# topleft, topcenter, topright, midleft, midcenter, midright, botleft, botcenter, botright
patch_ndc_points = [(1.0, 1.0, 1.0), (0.5, 1.0, 1.0), (0.0, 1.0, 1.0),
                    (1.0, 0.5, 1.0), (0.5, 0.5, 1.0), (0.0, 0.5, 1.0),
                    (1.0, 0.0, 1.0), (0.5, 0.0, 1.0), (0.0, 0.0, 1.0)]

In [24]:
patch_ndc_to_ndc_transform = get_patch_ndc_to_ndc_transform(cameras=cam, 
                                                            image_size=image_size, 
                                                            patch_size=patch_size, 
                                                            patch_center=patch_center)
ndc_points = patch_ndc_to_ndc_transform.transform_points(torch.tensor(patch_ndc_points))
ndc_points

tensor([[1.0000, 1.0000, 1.0000],
        [0.5000, 1.0000, 1.0000],
        [0.0000, 1.0000, 1.0000],
        [1.0000, 0.5000, 1.0000],
        [0.5000, 0.5000, 1.0000],
        [0.0000, 0.5000, 1.0000],
        [1.0000, 0.0000, 1.0000],
        [0.5000, 0.0000, 1.0000],
        [0.0000, 0.0000, 1.0000]])

In [25]:
patch_ndc_revert_transform = get_ndc_to_patch_ndc_transform(cameras=cam,
                                                            image_size=image_size, 
                                                            patch_size=patch_size, 
                                                            patch_center=patch_center)
patch_ndc_points_revert = patch_ndc_revert_transform.transform_points(ndc_points)
patch_ndc_points_revert

tensor([[1.0000, 1.0000, 1.0000],
        [0.5000, 1.0000, 1.0000],
        [0.0000, 1.0000, 1.0000],
        [1.0000, 0.5000, 1.0000],
        [0.5000, 0.5000, 1.0000],
        [0.0000, 0.5000, 1.0000],
        [1.0000, 0.0000, 1.0000],
        [0.5000, 0.0000, 1.0000],
        [0.0000, 0.0000, 1.0000]])

In [26]:
ndc_to_screen_transform = get_ndc_to_screen_transform(cameras=cam, image_size=image_size)
screen_points = ndc_to_screen_transform.transform_points(ndc_points)       
screen_points                                        

tensor([[   0.,    0.,    1.],
        [ -64.,    0.,    1.],
        [-128.,    0.,    1.],
        [   0.,  -64.,    1.],
        [ -64.,  -64.,    1.],
        [-128.,  -64.,    1.],
        [   0., -128.,    1.],
        [ -64., -128.,    1.],
        [-128., -128.,    1.]])

In [27]:
screen_to_world_transform = cam.unproject_points(screen_points)

In [28]:
screen_to_world_transform

tensor([[   0.,    0.,    1.],
        [ -64.,    0.,    1.],
        [-128.,    0.,    1.],
        [   0.,  -64.,    1.],
        [ -64.,  -64.,    1.],
        [-128.,  -64.,    1.],
        [   0., -128.,    1.],
        [ -64., -128.,    1.],
        [-128., -128.,    1.]])